In [14]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

import torch.optim as optim

import itertools
import random

import argparse
import tqdm


import torch
import numpy as np
from scipy import linalg

import glob
import cv2
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
from scipy.signal import find_peaks


## Recall Function
def top_k_recall(similarity_matrix, k):
    """
    Compute top-k recall from a similarity matrix (vectorized version).

    similarity_matrix: Tensor of shape (batch_size, batch_size), where
                       diagonal elements represent positive pairs.
    k: The value of k for top-k recall.

    Returns:
        recall: The top-k recall (mean recall across all examples).
    """
    batch_size = similarity_matrix.size(0)

    # Get the indices of the top-k most similar items for each row (video/audio example)
    _, top_k_indices = torch.topk(similarity_matrix, k, dim=1)

    # Create a tensor for diagonal indices (i, i) for each row
    diagonal_indices = torch.arange(batch_size, device=similarity_matrix.device)

    # Check if the diagonal index of each row is in the top-K indices of that row
    # `top_k_indices` is of shape (batch_size, k)
    is_true_positive_in_top_k = (top_k_indices == diagonal_indices.unsqueeze(1))

    # Calculate recall: for each row, check if the diagonal index is in the top-K
    recall_per_row = is_true_positive_in_top_k.any(dim=1).float()

    # Return the mean recall across all examples (rows)
    return recall_per_row.mean()


### FAD Function
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Calculate the Fréchet Distance between two multivariate Gaussians."""
    mu1 = mu1.cpu().numpy()
    mu2 = mu2.cpu().numpy()
    sigma1 = sigma1.cpu().numpy()
    sigma2 = sigma2.cpu().numpy()

    diff = mu1 - mu2

    covmean, _ = linalg.sqrtm(sigma1 @ sigma2, disp=False)
    if not np.isfinite(covmean).all():
        print("Adding epsilon to diagonal of covariance matrices for stability.")
        sigma1 += np.eye(sigma1.shape[0]) * eps
        sigma2 += np.eye(sigma2.shape[0]) * eps
        covmean = linalg.sqrtm(sigma1 @ sigma2)

    # Numerical error might give slight imaginary part
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    distance = (diff @ diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean
    return torch.tensor(distance, dtype=torch.float32)


def compute_fad(real_embeddings, retrieved_embeddings):
    """Compute FAD between real and generated embeddings."""
    mu_real = real_embeddings.mean(dim=0)
    sigma_real = torch.cov(real_embeddings.T)

    mu_gen = retrieved_embeddings.mean(dim=0)
    sigma_gen = torch.cov(retrieved_embeddings.T)

    fad = calculate_frechet_distance(mu_real, sigma_real, mu_gen, sigma_gen)
    return fad


def detect_video_peaks(frames, fps, threshold=1, distance=3):
    """
    Detect motion peaks using optical flow and find_peaks.
    
    Args:
        frames: list of grayscale frames
        fps: frames per second (adjusted for skipping)
        threshold: minimum peak height (motion strength)
        distance: minimum number of frames between peaks

    Returns:
        flow_trajectory: List of average flow magnitudes
        peak_times: Timestamps of detected peaks (in seconds)
    """
    distance = distance * fps
    # Compute optical flow magnitudes between consecutive frames
    flow_trajectory = [
        compute_of(frames[i - 1], frames[i])
        for i in range(1, len(frames))
    ]

    # Use scipy to detect peaks
    peak_indices, properties = find_peaks(
        flow_trajectory,
        height=threshold,       # Minimum motion strength
        distance=distance       # Minimum distance between peaks
    )

    # Convert frame indices to time
    peak_times = [idx / fps for idx in peak_indices]

    return flow_trajectory, peak_times


def extract_frames(video_path, frame_skip=5, downsample_factor=2, max_frames=360):
    """Extract grayscale, downsampled frames from video with frame skipping."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Error: Unable to open video {video_path}.")

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    max_frames = max_frames*original_fps
    frames = []
    frame_idx = 0

    while True:
        success, frame = cap.read()
        if not success:
            break
        if frame_idx % frame_skip == 0:
            if downsample_factor > 1:
                h, w = frame.shape[:2]
                frame = cv2.resize(frame, (w // downsample_factor, h // downsample_factor))
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frames.append(gray)
        frame_idx += 1
        if frame_idx > max_frames:
            break
    cap.release()

    effective_fps = original_fps / frame_skip
    return frames, effective_fps

def detect_audio_peaks(audio_file):
    waveform, sr = torchaudio.load(audio_file)
    waveform = waveform.mean(dim=0)  # Convert to mono

    # Compute the short-time energy
    frame_length = int(0.025 * sr)  # 25ms frames
    hop_length = int(0.010 * sr)    # 10ms hop

    energy = waveform.unfold(0, frame_length, hop_length).pow(2).mean(dim=1)

    # Normalize energy
    energy = (energy - energy.min()) / (energy.max() - energy.min() + 1e-6)

    # Detect peaks: simple threshold
    peaks = (energy[1:-1] > energy[:-2]) & (energy[1:-1] > energy[2:]) & (energy[1:-1] > 0.3)
    peak_indices = torch.nonzero(peaks).squeeze() + 1  # +1 because of slicing

    onset_times = (peak_indices * hop_length) / sr
    return onset_times.tolist()


# Function to find local maxima with threshold
def find_local_max_indexes(arr, fps):
    arr = torch.tensor(arr)
    local_max_mask = (arr[1:-1] > arr[:-2]) & (arr[1:-1] > arr[2:]) & (arr[1:-1] >= 0.5)
    indexes = torch.where(local_max_mask)[0] + 1  # offset by 1 due to slicing
    times = indexes.float() / fps
    return times.tolist()

# Function to compute the optical flow magnitude between two frames
def compute_of(prev_gray, curr_gray):
    flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None,
                                        0.5, 3, 15, 3, 5, 1.2, 0)
    magnitude = cv2.magnitude(flow[..., 0], flow[..., 1])
    avg_magnitude = magnitude.mean()
    return avg_magnitude

# Function to detect video peaks
def detect_video_peaks(frames, fps):
    flow_trajectory = [
        compute_of(frames[i-1], frames[i])
        for i in range(1, len(frames))
    ]
    video_peaks = find_local_max_indexes(flow_trajectory, fps)
    return flow_trajectory, video_peaks

# Function to calculate Intersection over Union (IoU)
def calc_intersection_over_union(audio_peaks, video_peaks, fps):
    intersection_length = 0
    used_video_peaks = [False] * len(video_peaks)
    for audio_peak in audio_peaks:
        for j, video_peak in enumerate(video_peaks):
            if not used_video_peaks[j] and abs(video_peak - audio_peak) <= (1 / fps):
                intersection_length += 1
                used_video_peaks[j] = True
                break
    union = len(audio_peaks) + len(video_peaks) - intersection_length
    return intersection_length / union if union > 0 else 0

In [2]:
test_path = '/work/users/s/m/smerrill/Youtube8m/test_resnet.csv'
vid_path = '/work/users/s/m/smerrill/Youtube8m/video'
aud_path = '/work/users/s/m/smerrill/Youtube8m/audio'

In [3]:
video_files = os.listdir(vid_path)
audio_files = os.listdir(aud_path)

In [4]:
# Open and read the CSV file
with open(test_path, 'r') as file:
    lines = file.readlines()
test_ids = [line.replace('.npy\n', '') for line in lines]

In [5]:
test_vid_files = []
for video_file in video_files:
    tmp = video_file.split('.')[0]
    
    if tmp in test_ids:
        test_vid_files.append(os.path.join(vid_path, video_file))

In [6]:
test_aud_files = []
for audio_file in audio_files:
    tmp = audio_file.split('.')[0]
    
    if tmp in test_ids:
        test_aud_files.append(os.path.join(aud_path, audio_file))

In [7]:
os.makedirs('/work/users/s/m/smerrill/SymMV/video_peaks/')
os.makedirs('/work/users/s/m/smerrill/SymMV/audio_peaks/')
os.makedirs('/work/users/s/m/smerrill/Youtube8m/video_peaks/')
os.makedirs('/work/users/s/m/smerrill/Youtube8m/audio_peaks/')

FileExistsError: [Errno 17] File exists: '/work/users/s/m/smerrill/SymMV/video_peaks/'

In [15]:
for i in range(len(video_files)):
    try:
        vid_file = test_vid_files[i]
        aud_file = test_aud_files[i]
        print(i, vid_file, aud_file)
        vid_save_path = vid_file.replace('video', 'video_peaks').split('.')[0] + '.npy'
        aud_save_path = aud_file.replace('audio', 'audio_peaks').split('.')[0] + '.npy'
        
        #if os.path.isfile(aud_save_path):
        #    print(f"SKIPPING {vid_file}")
        #    continue

        frames, fps = extract_frames(vid_file, frame_skip=5, downsample_factor=2)
        _, video_peaks = detect_video_peaks(frames, fps)
        audio_peaks = detect_audio_peaks(aud_file)
        
        np.save(vid_save_path, video_peaks)
        np.save(aud_save_path, audio_peaks)
        
    except Exception as e:
        print(i, e)
        continue

0 /work/users/s/m/smerrill/Youtube8m/video/--SM0mwlHjY.mp4 /work/users/s/m/smerrill/Youtube8m/audio/--SM0mwlHjY.mp3
1 /work/users/s/m/smerrill/Youtube8m/video/-63Da0zOWWo.mp4 /work/users/s/m/smerrill/Youtube8m/audio/-63Da0zOWWo.mp3
2 /work/users/s/m/smerrill/Youtube8m/video/-6jwBiRsWP0.mp4 /work/users/s/m/smerrill/Youtube8m/audio/-6jwBiRsWP0.mp3
3 /work/users/s/m/smerrill/Youtube8m/video/-AICqjB56gE.mp4 /work/users/s/m/smerrill/Youtube8m/audio/-AICqjB56gE.mp3
4 /work/users/s/m/smerrill/Youtube8m/video/-DNnvpMjq9g.mp4 /work/users/s/m/smerrill/Youtube8m/audio/-DNnvpMjq9g.mp3
5 /work/users/s/m/smerrill/Youtube8m/video/-IdR0ifDCUE.mp4 /work/users/s/m/smerrill/Youtube8m/audio/-IdR0ifDCUE.mp3
6 /work/users/s/m/smerrill/Youtube8m/video/-L-I2wzV5Ss.mp4 /work/users/s/m/smerrill/Youtube8m/audio/-L-I2wzV5Ss.mp3
7 /work/users/s/m/smerrill/Youtube8m/video/-MLECADky0c.mp4 /work/users/s/m/smerrill/Youtube8m/audio/-MLECADky0c.mp3
8 /work/users/s/m/smerrill/Youtube8m/video/-O_jcpg7Fns.mp4 /work/users/s


KeyboardInterrupt


KeyboardInterrupt



In [26]:
frames, fps = extract_frames(vid_file, frame_skip=2, downsample_factor=2)

_, video_peaks = detect_video_peaks(frames, fps)


In [27]:
video_peaks

[0.06666667014360428,
 0.4000000059604645,
 0.6000000238418579,
 0.8666666746139526,
 1.1333333253860474,
 1.2666666507720947,
 1.4666666984558105,
 1.6666666269302368,
 1.9333332777023315,
 2.200000047683716,
 2.3333332538604736,
 2.5999999046325684,
 2.799999952316284,
 3.0,
 3.3333332538604736,
 3.6666667461395264,
 3.799999952316284,
 4.0,
 4.266666889190674,
 4.4666666984558105,
 5.199999809265137,
 6.199999809265137,
 7.133333206176758,
 7.266666889190674,
 7.400000095367432,
 7.5333333015441895,
 7.666666507720947,
 7.866666793823242,
 8.133333206176758,
 8.399999618530273,
 8.533333778381348,
 8.666666984558105,
 8.933333396911621,
 9.133333206176758,
 9.399999618530273,
 9.600000381469727,
 9.866666793823242,
 10.066666603088379,
 10.399999618530273,
 10.600000381469727,
 10.866666793823242,
 11.0,
 11.399999618530273,
 11.600000381469727,
 11.733333587646484,
 11.866666793823242,
 12.933333396911621,
 13.199999809265137,
 13.800000190734863,
 14.066666603088379,
 14.399999618

In [28]:
frames, fps = extract_frames(vid_file, frame_skip=10, downsample_factor=2)

_, video_peaks = detect_video_peaks(frames, fps)


In [29]:
video_peaks

[0.3333333432674408,
 1.0,
 1.6666666269302368,
 2.6666667461395264,
 3.6666667461395264,
 4.333333492279053,
 6.333333492279053,
 7.333333492279053,
 9.0,
 10.0,
 11.0,
 13.0,
 13.666666984558105,
 15.0,
 15.666666984558105,
 16.33333396911621,
 17.66666603088379,
 18.33333396911621,
 19.33333396911621,
 20.33333396911621,
 21.0,
 22.33333396911621,
 24.0,
 25.33333396911621,
 27.33333396911621,
 28.33333396911621,
 30.33333396911621,
 31.0,
 31.66666603088379,
 32.66666793823242,
 34.0,
 35.33333206176758,
 36.33333206176758,
 37.66666793823242,
 39.0,
 39.66666793823242,
 42.0,
 43.0,
 43.66666793823242,
 45.33333206176758,
 46.0,
 46.66666793823242,
 47.66666793823242,
 49.0,
 49.66666793823242,
 50.33333206176758,
 52.0,
 53.33333206176758,
 54.33333206176758,
 55.33333206176758,
 56.33333206176758,
 57.0,
 57.66666793823242,
 59.0,
 60.0,
 61.66666793823242,
 62.66666793823242,
 63.33333206176758,
 64.0,
 64.66666412353516,
 65.66666412353516,
 66.66666412353516,
 69.333335876464

In [42]:
_, video_peaks = detect_video_peaks(frames, fps)


In [43]:
video_peaks

[np.float64(2.6666666666666665),
 np.float64(7.333333333333333),
 np.float64(11.0),
 np.float64(15.0),
 np.float64(20.333333333333332),
 np.float64(25.333333333333332),
 np.float64(28.333333333333332),
 np.float64(31.666666666666668),
 np.float64(36.333333333333336),
 np.float64(39.666666666666664),
 np.float64(45.333333333333336),
 np.float64(49.666666666666664),
 np.float64(54.333333333333336),
 np.float64(57.666666666666664),
 np.float64(61.666666666666664),
 np.float64(65.66666666666667),
 np.float64(72.0),
 np.float64(75.0),
 np.float64(78.66666666666667),
 np.float64(82.33333333333333),
 np.float64(86.66666666666667),
 np.float64(89.66666666666667),
 np.float64(95.0),
 np.float64(98.33333333333333),
 np.float64(103.0),
 np.float64(106.0),
 np.float64(111.66666666666667),
 np.float64(116.0),
 np.float64(119.0),
 np.float64(122.0),
 np.float64(126.0),
 np.float64(129.66666666666666),
 np.float64(135.0),
 np.float64(138.66666666666666),
 np.float64(143.0),
 np.float64(146.3333333333

In [46]:
np.load('/work/users/s/m/smerrill/SymMV/video_peaks/-BjZmE2gtdo.npy')

array([  0.41708335,   2.91958332,   3.75375009,   6.67333364,
         7.50750017,   8.75874996,  10.84416676,  11.67833328,
        13.34666729,  15.43208408,  16.26625061,  17.51749992,
        18.76875114,  20.02000046,  22.52250099,  24.19083405,
        25.8591671 ,  27.52750015,  28.36166763,  34.20083618,
        35.03499985,  40.04000092,  41.29125214,  42.12541962,
        43.79375076,  45.87916946,  47.13041687,  48.38166809,
        49.63291931,  51.30125046,  55.05500031,  56.72333527,
        57.55750275,  60.47708511,  62.97958374,  63.81375122,
        66.7333374 ,  68.40167236,  70.06999969,  70.90416718,
        71.73833466,  72.57250214,  73.40666962,  75.07500458,
        76.3262558 ,  77.16041565,  77.99458313,  80.49708557,
        82.16542053,  83.83375549,  85.91916656,  88.00458527,
        88.83875275,  90.50708771,  92.59250641,  93.84375   ,
        94.67791748,  95.51208496,  97.18041992,  98.0145874 ,
        99.26583862, 100.1000061 , 102.18541718, 104.27

In [20]:
pd.read_csv('/work/users/s/m/smerrill/Youtube8m/flow/ranks_400.csv').sort_values('vid')

,vid,ranks,segments,top_start,top_end,bottom_start,bottom_end
1253,--SM0mwlHjY,[7 5 1 3 4 8 2 6],"[0, 19, 42, 52, 74, 85, 95, 123, 133]",42.0,52.000000,85.0,95.000000
1036,-63Da0zOWWo,[8 5 2 1 6 3 4 7],"[0, 16, 32, 42, 52, 77, 94, 109, 133]",42.0,52.000000,0.0,16.000000
623,-6jwBiRsWP0,[4 8 6 1 5 2 7 3],"[0, 23, 42, 58, 69, 82, 96, 113, 133]",58.0,69.000000,23.0,42.000000
53,-AICqjB56gE,[4 8 1 5 7 2 6 3],"[0, 18, 28, 51, 76, 86, 108, 118, 133]",28.0,51.000000,18.0,28.000000
1248,-DNnvpMjq9g,[8 7 1 5 4 2 3 6],"[0, 16, 32, 47, 67, 77, 87, 109, 133]",32.0,47.000000,0.0,16.000000
...,...,...,...,...,...,...,...
841,zn2m84TeOps,[7 2 4 6 1 5 8 3],"[0, 11, 23, 48, 60, 79, 105, 115, 133]",60.0,79.000000,105.0,115.000000
76,znP-6iEYeF0,[8 7 2 6 3 4 5 1],"[0, 10, 23, 37, 47, 81, 94, 108, 133]",108.0,133.333333,0.0,10.000000
292,znsikHT6WRo,[4 7 3 8 2 5 1 6],"[0, 18, 28, 44, 56, 69, 84, 112, 133]",84.0,112.000000,44.0,56.000000
682,zpHtixFUX6o,[5 7 3 6 1 2 4 8],"[0, 22, 32, 54, 64, 74, 106, 118, 133]",64.0,74.000000,118.0,133.333333


In [27]:
test_path = '/work/users/s/m/smerrill/Youtube8m/test_resnet.csv'
vid_feature_dir = '/work/users/s/m/smerrill/Youtube8m/resnet/resnet101'
aud_feature_dir = '/work/users/s/m/smerrill/Youtube8m/vggish'
flow_dir = '/work/users/s/m/smerrill/Youtube8m/flow'

# Open and read the CSV file
with open(test_path, 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

In [31]:
def get_meta_df(video_feature_path, audio_feature_path, flow_rank_file, max_seq_len, test_file=None):
    def parse_ranks_str(ranks_str):
        # Remove tuple and quotes, then extract numbers
        cleaned = ranks_str.strip("(),'")  # removes parentheses, commas, quotes
        return list(map(int, cleaned.strip('[]').split()))


    video_files = os.listdir(video_feature_path)
    audio_files = os.listdir(audio_feature_path)

    video_file_map = {filename.split('.')[0]: os.path.join(video_feature_path, filename) for filename in video_files }
    audio_file_map = {filename.split('.')[0]: os.path.join(audio_feature_path, filename) for filename in audio_files}
    
    flow_ranks = pd.read_csv(flow_rank_file)
    flow_ranks_dict = dict(zip(flow_ranks['vid'], flow_ranks['ranks']))
    
    vid_df = pd.DataFrame(list(video_file_map.items()), columns=['vid', 'vid_filename'])
    aud_df = pd.DataFrame(list(audio_file_map.items()), columns=['vid', 'aud_filename'])
    flow_ranks = pd.read_csv(flow_rank_file)[['vid', 'ranks', 'segments']]

    df = vid_df.merge(aud_df, on='vid').merge(flow_ranks, on='vid')

    df['ranks'] = df['ranks'].apply(
        lambda x: parse_ranks_str(x) if isinstance(x, str) else list(map(int, x)))

    # drop entire videos who's max sequence length doesn't comply
    df = df[df['segments'].apply(lambda s: max(np.diff(ast.literal_eval(s))) <= max_seq_len)].reset_index(drop=True)

    if test_file:
        # Open and read the CSV file
        with open(test_path, 'r') as file:
            lines = file.readlines()

        vids = [line.split('.')[0] for line in lines]

    return df
import ast

test = get_meta_df(vid_feature_dir, \
                  aud_feature_dir, \
                  '/work/users/s/m/smerrill/Youtube8m/flow/ranks.csv',\
                  200)

In [33]:
test

,vid,vid_filename,aud_filename,ranks,segments
0,--81RtUMW1w,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/--81...,"[3, 6, 2, 4, 1, 8, 5, 7]","[0, 35, 49, 68, 87, 103, 113, 123, 146]"
1,--LxRKErLk8,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/--Lx...,"[6, 3, 5, 2, 4, 7, 1, 8]","[0, 10, 93, 129, 155, 165, 175, 232, 242]"
2,--SM0mwlHjY,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/--SM...,"[8, 3, 6, 4, 7, 1, 5, 2]","[0, 42, 52, 95, 125, 144, 181, 202, 232]"
3,--pYDRqWOhk,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/--pY...,"[3, 7, 2, 6, 4, 1, 5, 8]","[0, 70, 82, 115, 130, 193, 205, 244, 254]"
4,--v2_eyl-xg,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/--v2...,"[7, 3, 6, 1, 4, 2, 5, 8]","[0, 21, 178, 189, 199, 280, 308, 318, 328]"
...,...,...,...,...,...
15668,zy65W7be-T4,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/zy65...,"[5, 8, 7, 2, 4, 3, 1, 6]","[0, 24, 52, 93, 103, 141, 187, 207, 232]"
15669,zzHj0MMhZRM,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/zzHj...,"[5, 3, 8, 2, 7, 1, 4, 6]","[0, 82, 104, 114, 125, 155, 165, 235, 278]"
15670,zzWPUaKVg7c,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/zzWP...,"[6, 4, 1, 2, 5, 3, 7, 8]","[0, 17, 33, 47, 112, 124, 139, 149, 159]"
15671,zzWZchxa_Ws,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/zzWZ...,"[6, 2, 7, 1, 5, 3, 4, 8]","[0, 51, 75, 86, 107, 145, 193, 307, 317]"


In [34]:
with open(test_path, 'r') as file:
    lines = file.readlines()

vids = [line.split('.')[0] for line in lines]


In [37]:
test[test.vid.isin(vids)].reset_index(drop=True)

,vid,vid_filename,aud_filename,ranks,segments
2,--SM0mwlHjY,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/--SM...,"[8, 3, 6, 4, 7, 1, 5, 2]","[0, 42, 52, 95, 125, 144, 181, 202, 232]"
37,-63Da0zOWWo,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/-63D...,"[4, 3, 6, 1, 7, 2, 5, 8]","[0, 52, 109, 129, 140, 155, 210, 230, 242]"
38,-6jwBiRsWP0,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/-6jw...,"[4, 8, 5, 3, 6, 1, 2, 7]","[0, 23, 42, 58, 96, 113, 145, 209, 223]"
56,-AICqjB56gE,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/-AIC...,"[4, 8, 2, 6, 3, 7, 1, 5]","[0, 18, 28, 76, 86, 143, 153, 169, 183]"
65,-DNnvpMjq9g,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/-DNn...,"[7, 5, 2, 3, 1, 4, 6, 8]","[0, 16, 32, 109, 147, 157, 188, 198, 210]"
...,...,...,...,...,...
15624,zn2m84TeOps,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/zn2m...,"[7, 1, 4, 6, 3, 5, 2, 8]","[0, 11, 23, 79, 115, 134, 177, 213, 246]"
15628,znP-6iEYeF0,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/znP-...,"[8, 7, 5, 3, 4, 2, 6, 1]","[0, 10, 23, 81, 94, 108, 130, 228, 238]"
15629,znsikHT6WRo,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/znsi...,"[3, 5, 7, 4, 1, 6, 2, 8]","[0, 18, 44, 56, 87, 112, 161, 253, 263]"
15633,zpHtixFUX6o,/work/users/s/m/smerrill/Youtube8m/resnet/resn...,/work/users/s/m/smerrill/Youtube8m/vggish/zpHt...,"[5, 3, 6, 1, 2, 7, 4, 8]","[0, 28, 54, 64, 74, 118, 143, 190, 204]"
